In [1]:
import os
import requests
from bs4 import BeautifulSoup

There were a few APIs at the time of this notebook.
Directly at pypi
- https://wiki.python.org/moin/PyPIJSON
- In the format https://pypi.python.org/pypi/<package_name>/json

There is also https://pypistats.org/
 - They ask that any large amount of API access go through Google Big Query

Google Big Query
- https://cloud.google.com/docs/authentication/getting-started
- https://bigquery.cloud.google.com/table/the-psf:pypi.downloads


In [30]:
# May need to install these
#!pip install jk_pypiorgapi
#!pip install pyarrow

In [3]:
import jk_pypiorgapi

api = jk_pypiorgapi.PyPiOrgAPI()
n = len(api.listAllPackages())
print("Number of packages on pypi.org:", n)

Number of packages on pypi.org: 330133


In [4]:
temp = api.listAllPackages()
for i in range(10):
    print(temp[i])

('/simple/0/', '0')
('/simple/0-0/', '0-._.-._.-._.-._.-._.-._.-0')
('/simple/00000a/', '00000a')
('/simple/0-0-1/', '0.0.1')
('/simple/00print-lol/', '00print_lol')
('/simple/00smalinux/', '00SMALINUX')
('/simple/0121/', '0121')
('/simple/01changer/', '01changer')
('/simple/01d61084-d29e-11e9-96d1-7c5cf84ffe8e/', '01d61084-d29e-11e9-96d1-7c5cf84ffe8e')
('/simple/01-distributions/', '01-distributions')


In [30]:
#print(type(temp))
prj_str = 'tzflaskoidc'
matches = [match for match in temp if prj_str in match]

print(matches)

[('/simple/tzflaskoidc/', 'tzflaskoidc')]


## Authenticate and Construct Google Big Query Object

In [7]:
from google.cloud import bigquery
from google.oauth2 import service_account

# TODO(developer): Set key_path to the path to the service account key
#                  file.
key_path = "msds-pypi-89bc81421c55.json"

credentials = service_account.Credentials.from_service_account_file(
    key_path, scopes=["https://www.googleapis.com/auth/cloud-platform"],
)

client = bigquery.Client(credentials=credentials, project=credentials.project_id,)

# Construct a BigQuery client object.
#client = bigquery.Client()

query = """
    SELECT COUNT(*) AS num_downloads
    FROM `bigquery-public-data.pypi.file_downloads`
    WHERE file.project = 'pytest'
    -- Only query the last 30 days of history
    AND DATE(timestamp)
    BETWEEN DATE_SUB(CURRENT_DATE(), INTERVAL 30 DAY)
    AND CURRENT_DATE()
"""
query_job = client.query(query)  # Make an API request.

print("The query data:")
results = query_job.result()  # Waits for job to complete.
for row in results:
    print("{} downloads".format(row.num_downloads))

The query data:
32746206 downloads


In [8]:
print(query_job)
for row in query_job:
    print(row)

QueryJob<project=msds-pypi, location=US, id=d2899f0b-ab2b-434e-993f-74f8ffeacd4b>
Row((32746206,), {'num_downloads': 0})


In [27]:
query_job2 = client.query("""
SELECT name,version,license
FROM `bigquery-public-data.pypi.distribution_metadata`
WHERE name = 'atlas'""")

results2 = query_job2.result()  # Waits for job to complete.
#for row in results2:
    #print("{} downloads".format(row.num_downloads))
 #   print(row)

In [28]:
for row in results2:
    #print("{} downloads".format(row.num_downloads))
    print(row)

Row(('atlas', '0.25', 'Apache 2.0'), {'name': 0, 'version': 1, 'license': 2})
Row(('atlas', '0.25.6', 'Apache 2.0'), {'name': 0, 'version': 1, 'license': 2})
Row(('atlas', '0.25.4', None), {'name': 0, 'version': 1, 'license': 2})
Row(('atlas', '0.25.2', 'Apache 2.0'), {'name': 0, 'version': 1, 'license': 2})
Row(('atlas', '0.25.3', 'Apache 2.0'), {'name': 0, 'version': 1, 'license': 2})
Row(('atlas', '0.25.5', None), {'name': 0, 'version': 1, 'license': 2})
Row(('atlas', '0.21', 'Apache 2.0'), {'name': 0, 'version': 1, 'license': 2})
Row(('atlas', '0.25.5', None), {'name': 0, 'version': 1, 'license': 2})
Row(('atlas', '0.25.1', 'Apache 2.0'), {'name': 0, 'version': 1, 'license': 2})
Row(('atlas', '0.25.3', 'Apache 2.0'), {'name': 0, 'version': 1, 'license': 2})
Row(('atlas', '0.25.3', 'Apache 2.0'), {'name': 0, 'version': 1, 'license': 2})
Row(('atlas', '0.27.0', 'Apache 2.0'), {'name': 0, 'version': 1, 'license': 2})
Row(('atlas', '0.25.4', None), {'name': 0, 'version': 1, 'license': 

In [35]:
from google.cloud import bigquery

bqclient = bigquery.Client(credentials=credentials, project=credentials.project_id)

# Download a table.
table = bigquery.TableReference.from_string(
    "bigquery-public-data.pypi.distribution_metadata"
)
rows = bqclient.list_rows(
    table,
    selected_fields=[
        bigquery.SchemaField("name", "STRING"),
        bigquery.SchemaField("license", "STRING"),
    ],
)
dataframe = rows.to_dataframe(
    # Optionally, explicitly request to use the BigQuery Storage API. As of
    # google-cloud-bigquery version 1.26.0 and above, the BigQuery Storage
    # API is used by default.
    #create_bqstorage_client=True,
)
print(dataframe.head())

ValueError: The pyarrow library is not installed, please install pyarrow to use the to_arrow() function.

In [ ]:
query_job = client.query("""
SELECT COUNT(*) AS num_downloads
FROM `bigquery-public-data.pypi.file_downloads`
WHERE file.project = 'pytest'
  -- Only query the last 30 days of history
  AND DATE(timestamp)
    BETWEEN DATE_SUB(CURRENT_DATE(), INTERVAL 30 DAY)
    AND CURRENT_DATE()""")

results = query_job.result()  # Waits for job to complete.